In [1]:
import numpy as np
import os
import pandas as pd
import pickle
import seaborn as sns

from collections import OrderedDict
from pyinstrument import Profiler
from timeit import default_timer as timer

import dynml as dml

np.random.seed(42)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

%matplotlib notebook 

In [ ]:
%load_ext rpy2.ipython

# Set Inputs

In [ ]:
bnfuncsfile = '_swarmout/gbmbnet_wm_raf_ck2_0/gbmbnet_wm_raf_ck2_0.txt'
context = 'WM'
#------------------------------------------------------------------------------------------
assert(os.path.isfile(bnfuncsfile))
bnfuncsdir = os.path.dirname(bnfuncsfile)
dfsdir = bnfuncsdir + '/dfs/'
input_to_attr_df_file = bnfuncsdir + '/input_to_attr_df.pkl'
uniq_attr_df_file = bnfuncsdir + '/uniq_attr_df.pkl'

# set inputs based on context
# below: check phenotype column based on context
if context == 'BP':
    input_nodes = ['EGF', 'PDGF', 'WNT_Canonical', 'TIMP', 'DNA_Damage', 'Ephrin_B1_B2', 'Oxygen']
elif context == 'PS':
    input_nodes = ['EGF', 'PDGF', 'WNT_Canonical', 'TIMP', 'DNA_Damage', 'Ephrin_B1_B2']
elif context == 'WM':
    input_nodes = ['EGF', 'PDGF', 'WNT_Canonical', 'TIMP', 'DNA_Damage', 'Ephrin_B1_B2', 'Oxygen']
else:
    assert(False)

# Merge Biowulf Results

In [ ]:
dfs = [pd.read_pickle(dfsdir + '/' + pklfile) for pklfile in os.listdir(dfsdir) 
       if pklfile.endswith('.pkl')]
dfmerged = pd.concat(dfs)
dfmerged.insert(loc=0, column='_attr_str_', value=list(dfmerged.index))
assert(all(dfmerged.index == dfmerged._attr_str_))

d = {k:v for k, v in dfmerged.groupby(['__input_id__', '_attr_str_'])}

in_attr_dfs = []
for df in d.values():
    assert(df.shape[0] == 3)
    tmp = df.copy()
    tmp.__count__ = sum(tmp.__count__)
    tmp.drop_duplicates(inplace=True)
    assert(tmp.shape[0] == 1)
    in_attr_dfs.append(tmp)
    
input_attr_df = pd.concat(in_attr_dfs)
input_attr_df.sort_values(by=['__input_id__', '__count__'], ascending=[True, False], inplace=True)
assert(len(input_attr_df.index) == len(set(input_attr_df.index)))
assert(all(input_attr_df.index == input_attr_df._attr_str_))
input_attr_df.drop(columns='_attr_str_', inplace=True)

if context == 'BP':
    assert(all(input_attr_df.BRAIN_PARENCHYMA.astype(bool)))
elif context == 'PS':
    assert(all(input_attr_df.PERIVASCULAR_SPACE.astype(bool)))
elif context == 'WM':
    assert(all(input_attr_df.WHITE_MATTER_TRACT.astype(bool)))
else:
    # Unexpected!
    assert(False)

# Attractor Verification

In [3]:
node_to_func_str, _ = dml.read_boolean_functions(bnfuncsfile)
bool_funcs = {k : dml.get_boolean_function(v) for (k,v) in node_to_func_str.items()}
#len([(len(v) if (type(v) is list) else 1) for k,v in tmp.items()])

In [ ]:
tmp = input_attr_df.drop(columns=['__input_id__', '__count__']).copy()
false_attractor_indices = []

for i in range(tmp.shape[0]):
    print(str(i) + " ", end='')
    if (i+1 % 40) == 0: print("\n", end='')
    if any(pd.isna(tmp.iloc[i, :])): continue
    if not dml.is_attractor_state(state=tmp.iloc[i, :], inputs=tmp.iloc[i, :8], 
                                  bool_funcs=bool_funcs, n_steps=10000):
        false_attractor_indices.append(i)

assert(not false_attractor_indices)

# Assess Phenotype Distribution

In [ ]:
def get_phenotype_table(uniq_attr_df):
    tmp = uniq_attr_df.copy()
    tmp['Dormant'] = np.nan
    for attr in tmp.index:
        cyc = tmp.loc[attr, 'Cell_Cycle']
        mot = tmp.loc[attr, 'Directed_Motility']
        apo = tmp.loc[attr, 'Apoptosis']
        if np.isnan(cyc) or np.isnan(mot) or np.isnan(apo):
            continue
        cyc = bool(cyc)
        mot = bool(mot)
        apo = bool(apo)
        tmp.loc[attr, 'Dormant'] = int((not cyc) and (not mot) and (not apo))

    #var_set = ['Directed_Motility', 'Cell_Cycle', 'Apoptosis', 'Dormant', 'pOLIG2']
    var_set = ['Directed_Motility', 'Cell_Cycle', 'Apoptosis', 'Dormant', 'pOLIG2', 'HIF', 'SMAD_2_3_4']
    pw_counts = dml.get_pairwise_true_counts(tmp, var_set)
    tmp = pw_counts.loc[var_set[:4], var_set[:4]].values
    assert(np.count_nonzero(tmp - np.diag(np.diagonal(tmp))) == 0)
    assert(pw_counts.loc['pOLIG2', 'Cell_Cycle'] == pw_counts.loc['pOLIG2', 'pOLIG2'])
    
    return pw_counts

In [ ]:
tmp = input_attr_df.drop(columns=['__input_id__', '__count__']).copy()
assert(input_nodes == list(tmp.columns[:len(input_nodes)]))
attr_nodes = sorted(list(set(tmp.columns) - set(input_nodes)))
assert('_noinput_attr_id_' not in input_attr_df.columns)

#----------------------------------------------------------------------------------------------------
attr_df  = tmp.loc[:, attr_nodes].copy()
attr_df.dropna(inplace=True)
attr_id_strings = attr_df.apply(dml.binary_state_to_str, axis=1)
attr_str_to_id = {attr_str : (attr_id + 1) 
                  for attr_id, attr_str in enumerate(sorted(attr_id_strings.unique()))}
attr_df.insert(loc=0, column='_noinput_attr_id_', value=[attr_str_to_id[s]  for s in attr_id_strings])
#----------------------------------------------------------------------------------------------------
uniq_attr_df = attr_df.copy()
uniq_attr_df.drop_duplicates(inplace=True)
uniq_attr_df.index = uniq_attr_df['_noinput_attr_id_'].values
uniq_attr_df.sort_values(by='_noinput_attr_id_', inplace=True)
uniq_attr_df = uniq_attr_df.astype(int)
#----------------------------------------------------------------------------------------------------

get_phenotype_table(uniq_attr_df)

In [ ]:
input_attr_df.to_pickle(path=input_to_attr_df_file)
uniq_attr_df.to_pickle(path=uniq_attr_df_file)

# View Interactive Attractor Data Tables

In [ ]:
%%R -i uniq_attr_df

library(DT)

DT::datatable(uniq_attr_df, rownames= TRUE, filter = 'top', class = 'cell-border stripe', 
              extensions = 'FixedHeader', 
              options = list(lengthMenu = c(64, 128, 256, 512), fixedHeader = TRUE)) 

In [ ]:
%%R -i input_attr_df

library(DT)

DT::datatable(input_attr_df, rownames= FALSE, filter = 'top', class = 'cell-border stripe', 
              extensions = 'FixedHeader', 
              options = list(lengthMenu = c(64, 128, 256, 512), fixedHeader = TRUE)) 

# Make Attractor Phenotype Distribution Summary Figure

In [ ]:
d = OrderedDict()
d['Unperturbed'] = '_swarmout/gbmbnet_wm/uniq_attr_df.pkl'
d['EGFR_0'] = '_swarmout/gbmbnet_wm_egfr_0/uniq_attr_df.pkl'
d['SRC_0'] = '_swarmout/gbmbnet_wm_src_0/uniq_attr_df.pkl'
d['MEK_0'] = '_swarmout/gbmbnet_wm_mek_0/uniq_attr_df.pkl'
d['TFGBR_0'] = '_swarmout/gbmbnet_wm_tgfbr_0/uniq_attr_df.pkl'
d['CK2_0'] = '_swarmout/gbmbnet_wm_ck2_0/uniq_attr_df.pkl'
#d['MEK_TGFBR_0'] = '_swarmout/bnfuncs_5_mek_tgfbr_0/uniq_attr_df.pkl'
#d['RAF_CK2_0'] = '_swarmout/bnfuncs_5_raf_ck2_0/uniq_attr_df.pkl'

bnattr_pheno = pd.DataFrame(-1, index=d.keys(), 
    columns=['Go', 'Grow (w/o pOlig2)', 'Grow (pOlig2)', 'Apoptosis', 'Dormant']
)

for nw, path in d.items():
    ptab = get_phenotype_table(pd.read_pickle(path))
    pvec = pd.Series(np.diag(ptab), index=ptab.columns)
    bnattr_pheno.loc[nw, 'Go'] = pvec['Directed_Motility']
    bnattr_pheno.loc[nw, 'Grow (w/o pOlig2)'] = pvec['Cell_Cycle'] - pvec['pOLIG2']
    bnattr_pheno.loc[nw, 'Grow (pOlig2)'] = pvec['pOLIG2']
    bnattr_pheno.loc[nw, 'Apoptosis'] = pvec['Apoptosis']
    bnattr_pheno.loc[nw, 'Dormant'] = pvec['Dormant']
    
bnattr_pheno_frac = bnattr_pheno.div(bnattr_pheno.sum(axis=1), axis=0)

In [ ]:
#http://benalexkeen.com/bar-charts-in-matplotlib/
import matplotlib.pyplot as plt

barplot_bottoms = np.cumsum(bnattr_pheno_frac, axis=1)
barplot_bottoms.insert(loc=0, column='tmp', value=0)
barplot_bottoms = barplot_bottoms.iloc[:, :5]
barplot_bottoms.columns = bnattr_pheno.columns

iset = list(range(bnattr_pheno_frac.shape[0]))
nwnames = list(bnattr_pheno_frac.index)
colors = ['g', 'b', 'cyan', 'r', 'orange']

for j, phtype in enumerate(bnattr_pheno_frac.columns):
    plt.bar(iset, bnattr_pheno_frac[phtype], label=phtype, 
            color=colors[j], bottom=barplot_bottoms[phtype])

plt.xticks(iset, nwnames)
plt.ylabel("Cumulative Proportion")
plt.xlabel("Boolean Networks")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("Attractor Phenotype Distribution")
plt.setp(plt.gca().get_xticklabels(), rotation=45, horizontalalignment='right')

plt.show()

# Make Biowulf Swarm File

In [ ]:
bnfilepath = '_swarmout/gbmbnet_bp_alt/gbmbnet_bp.txt'
N = 2**len(['EGF', 'PDGF', 'WNT_Canonical', 'TIMP', 'DNA_Damage', 'Ephrin_B1_B2', 'Oxygen'])
filename = '_bnsim_bp.swarm'
cmd = 'python swarm_bnsim_bp.py'

In [ ]:
bnfilepath = '_swarmout/gbmbnet_ps_alt/gbmbnet_ps.txt'
N = 2**len(['EGF', 'PDGF', 'WNT_Canonical', 'TIMP', 'DNA_Damage', 'Ephrin_B1_B2'])
filename = '_bnsim_ps.swarm'
cmd = 'python swarm_bnsim_ps.py'

In [ ]:
bnfilepath = '_swarmout/gbmbnet_wm_alt/gbmbnet_wm.txt'
N = 2**len(['EGF', 'PDGF', 'WNT_Canonical', 'TIMP', 'DNA_Damage', 'Ephrin_B1_B2', 'Oxygen'])
filename = '_bnsim_wm.swarm'
cmd = 'python swarm_bnsim_wm.py'

In [ ]:
input_indices = list(range(N))
random_seeds = [1, 2, 3]
num_init_conds = [3333, 3333, 3334]
assert(len(random_seeds) == len(num_init_conds))

with open(filename, 'wt') as f:
    for i in input_indices:
        for j, k in zip(random_seeds, num_init_conds):
            ln = cmd + ' ' + str(i) + ' ' + str(j) + ' ' + str(k) + ' ' + bnfilepath
            print(ln, file=f)
            

# Attractor to Expression Data Comparison

In [ ]:
node_to_genes = dml.read_node_to_gene_symbols_file('./data/bnnode_to_gene_symbols_reka.txt')
tmp_dict = dml.read_node_to_gene_symbols_file('./data/bnnode_to_gene_symbols.txt')
for k, v in tmp_dict.items():
    if k not in node_to_genes:
        node_to_genes[k] = v
        
ivygap = pd.read_pickle('inst/rawdata/compare_w_expdata/ivygap.pkl')
ivygap_exp = ivygap['exp']
ivygap_samples = ivygap['samples']

ivygap_node_exp = dml.get_boolnet_node_expression(ivygap_exp, node_to_genes).dropna().T

In [ ]:
# Color by Stem Status.
ivygap_samples_colors = pd.Series('black', index=ivygap_samples.index)
ivygap_samples_colors[ivygap_samples['stem_cluster_status'] == 'Stem Cluster'] = 'red'

In [ ]:
ivygap_samples_colors = pd.Series('yellow', index=ivygap_samples.index) # Cellular Tumor
ivygap_samples_colors[ivygap_samples['location'] == 'Pseudopalisading Cells'] = 'red'
ivygap_samples_colors[ivygap_samples['location'] == 'Microvascular Proliferation'] = 'green'
ivygap_samples_colors[ivygap_samples['location'] == 'Perinecrotic Zone'] = 'cyan'
ivygap_samples_colors[ivygap_samples['location'] == 'Infiltrating Tumor'] = 'magenta'
ivygap_samples_colors[ivygap_samples['location'] == 'Hyperplastic Blood Vessels'] = 'orange'
ivygap_samples_colors[ivygap_samples['location'] == 'Leading Edge'] = 'blue'

In [ ]:
np.random.seed(1)
assert(ivygap_node_exp.index is ivygap_samples_colors.index)
cmout_ivygap_nodexp = sns.clustermap(ivygap_node_exp, metric='euclidean', cmap='RdBu_r',
    method='complete', row_colors=ivygap_samples_colors, figsize=(20,12))

row_reord = ivygap_node_exp.index[cmout_ivygap_nodexp.dendrogram_row.reordered_ind]
col_reord = ivygap_node_exp.columns[cmout_ivygap_nodexp.dendrogram_col.reordered_ind]
ivygap_node_exp_reord = ivygap_node_exp.loc[row_reord, col_reord].copy()

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

from plotly.tools import set_credentials_file

set_credentials_file(username='vinodh.rajapakse', api_key='mQXgwB5B19HPXfbq2cvt')

trace = go.Heatmap(z=ivygap_node_exp_reord.values[::-1, :],
                   x=col_reord)
data=[trace]

layout = go.Layout(
    autosize=False,
    width=1400,
    height=800
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='labelled-heatmap')


# Run Stable Motifs Code (Locally)

In [ ]:
# Path to Boolean functions file (extracted from above PathVisio file).
boolean_functions_path = "./models/gbm_bnfuncs_2.txt"

# Path to Albert et al. stable motif (boolean network attractor) analysis code.
stable_motifs_lib_path = "/Users/rajapaksevn/repos/gbm_motility/lib/StableMotifs/dist/"

In [ ]:
bool_func_strings, _ = dml.read_boolean_functions(boolean_functions_path)
inputs = ['EGF', 'PDGF', 'WNT_Canonical', 'TIMP', 'DNA_Damage', 'Ephrin_B1_B2', 'Oxygen']

input_attr_df = dml.run_stable_motifs_for_input_combinations(inputs, bool_func_strings, stable_motifs_lib_path, 
                                                   timeout=300, output_file='bn_test_df_2.pkl')

# Approximate Input/Attractor Table by Simulation

In [ ]:
node_to_func_str, _ = dml.read_boolean_functions(bnfuncsfile)
bool_funcs = {k : dml.get_boolean_function(v) for (k,v) in node_to_func_str.items()}

node_list = input_nodes + sorted(list(set(bool_funcs.keys()) - set(input_nodes)))

np.random.seed(13)
start = timer()
input_attr_df = dml.get_simulation_based_input_attractor_tab(
    inputs=dml.get_state_table(pd.Series(np.nan, index=input_nodes)).iloc[:2,:], 
    nodes=node_list, 
    bool_funcs=bool_funcs, 
    n_steps=10000, 
    n_for_steady_state=5000, 
    n_init_conds=4, 
    synch_update=False, 
    outfile_root=None, 
    verbose=True)
end = timer()

print("\nElapsed Time: " + str(np.round(end - start, decimals=2)) + ' seconds')

# Profile Function

In [ ]:
start_state = pd.Series(np.nan, index=node_list)
start_state[input_nodes] = input_attr_df.iloc[100, :8]
start_state[non_input_nodes] = np.random.randint(low=0, high=2, size=len(non_input_nodes))

profiler = Profiler()
profiler.start()
traj = dml.get_boolnet_trajectory(init_state=start_state, inputs=start_state[input_nodes], 
                                  bool_funcs=bool_funcs, n_steps=10000, synch_update=False)
profiler.stop()
print(profiler.output_text(color=True))

#### Create Symbolic Link (ARG1:SOURCE ARG2:LINK, RESULT: LINK --> SOURCE)
ln -s /local/data/rajapaksevn/ data

In [ ]:
import dynml as dml
import numpy as np
import pandas as pd

input_index = 1
rseed = 13
boolean_functions_path = "_swarmout/bnfuncs_6/gbm_bnfuncs_6.txt"
nsteps = 1000
synch_update = False

node_to_func_str, _ = dml.read_boolean_functions(boolean_functions_path)
bool_funcs = {k : dml.get_boolean_function(v) for (k,v) in node_to_func_str.items()}

input_nodes = ['EGF', 'PDGF', 'WNT_Canonical', 'TIMP', 'ECM_Migratory_Stimuli', 'Oxygen', 'Physical_Barrier', 'Bradykinin']
non_input_nodes = sorted(list(set(bool_funcs.keys()) - set(input_nodes)))
node_list = input_nodes + non_input_nodes
input_tab = dml.get_state_table(pd.Series(np.nan, index=input_nodes))

start_state = pd.Series(np.nan, index=node_list)
start_state[input_nodes] = input_tab.iloc[input_index, :]

np.random.seed(rseed)
start_state[non_input_nodes] = np.random.randint(low=0, high=2, size=len(non_input_nodes))
np.random.seed(rseed+1)
traj = dml.get_boolnet_trajectory(init_state=start_state, inputs=start_state[input_nodes], 
    bool_funcs=bool_funcs, n_steps=nsteps, synch_update=synch_update)